In [1]:
!export TESSDATA_PREFIX=/usr/share/tessdata

In [2]:
import re
import fitz
import pickle
import pandas as pd
from logutils import get_logger
from selenium.webdriver.common.by import By
from exam_question_downloader.scraping.tools import initialize_webdriver, incremental_retry, do_with_delayed_retry, match_patterns
from exam_question_downloader.scraping.navigation import magic_directories_journey
from exam_question_downloader.utils.consts import SOURCE_URLS, OUTPUT_PATH, ROOT_DIR, SUBJECTS_PATTERNS
from exam_question_downloader.utils.text_processing import get_text_docs_from_dir, extract_questions_from_doc, classify_questions_by_subject

In [3]:
logger = get_logger("YEA")

In [4]:
ASSETS_DIR = ROOT_DIR.joinpath("assets")
ASSETS_DIR.mkdir(exist_ok=True)

# Scraping
Fase de scraping de las páginas donde la UPV publica las convocatorias de exámenes y los ejercicios resueltos para descargar los PDFs

In [ ]:
with initialize_webdriver(use_browser=True) as browser:
    for url in SOURCE_URLS:
        magic_directories_journey(url, browser, output_path=OUTPUT_PATH)

# Text processing

In [5]:
textdocs = get_text_docs_from_dir(OUTPUT_PATH)
ASSETS_DIR.joinpath("textdocs.pkl").write_bytes(pickle.dumps(textdocs))

# textdocs = pickle.loads(ASSETS_DIR.joinpath("textdocs.pkl").read_bytes())
# textdocs

[('71d1195a-2122-4366-a22f-3089accdf727.pdf',
  " \nACUERDO del tribunal de selección de las pruebas selectivas de acceso al grupo A, subgrupo \nA2, sector administración especial, categoría de ayudante de archivos, biblioteca y museos, \npor el sistema de concurso-oposición (Código: 2022/P/FC/C/1),  \n \n \nReunido el órgano técnico de selección se acuerda el cambio de lugar de celebración del \nprimer ejercicio de estas pruebas selectivas, que tendrá lugar el 30 de junio de 2023, a las 10:00 \nhoras. La nueva localización será: \n \n \n \nLugar de realización del primer ejercicio \nAula 234 de la Escuela Técnica Superior de \nIngeniería Industrial (Edificio 5H) \n \n \n \n \n \n \n \nValencia, a 15 de junio de 2023 \nLa presidenta del tribunal de selección. Dª. Inmaculada Romero Gil \n \n \n \n \nINMACULADA\n|ROMERO|GIL\nFirmado digitalmente por \nINMACULADA|ROMERO|GIL \nFecha: 2023.06.15 14:16:22 \n+02'00'\n"),
 ('82c7933d-8ccf-4469-9c77-d9336af34d7e.pdf',
  'PRIMER EJERCICIO – DFI 

In [6]:
# questiondocs = []
# for name, doc in textdocs:
#     questiondocs.append((name, extract_questions_from_doc(doc)))
# ASSETS_DIR.joinpath("questiondocs.pkl").write_bytes(pickle.dumps(questiondocs))

questiondocs = pickle.loads(ASSETS_DIR.joinpath("questiondocs.pkl").read_bytes())
questiondocs

[('71d1195a-2122-4366-a22f-3089accdf727.pdf', []),
 ('82c7933d-8ccf-4469-9c77-d9336af34d7e.pdf', []),
 ('b0a998a6-7e92-4c11-9ff5-224545bb9d48.pdf', []),
 ('2769253d-c7f3-4511-acce-54d0e089d1eb.pdf', []),
 ('8679bd3a-c94b-4a41-a2e4-b0c014c28824-ocr.pdf',
  [{'number': '1',
    'question': 'Según el artículo 1.1 de la Constitución Española, España seconstituye en:',
    'a': 'Un Estado liberal y democrático de Derecho.',
    'b': 'Un Estado autonómicoy social de Derecho.',
    'c': 'Un Estado social y democrático de Derecho.',
    'd': 'Un Estado social y liberal de Derecho.'},
   {'number': '2',
    'question': 'SegúnlaConstitución Española ¿Que tipo de enseñanza es obligatoria y gratuita?',
    'a': 'La enseñanza infantil.',
    'b': 'La enseñanza básica.',
    'c': 'La formación profesional.',
    'd': 'Toda la educación, excepto la universitaria.'},
   {'number': '3',
    'question': 'Según el Estatuto de Autonomía de la Comunitat Valenciana ¿Cuál de las siguientes es una institución

In [7]:
df = classify_questions_by_subject(questiondocs)
df

,number,question,a,b,c,d,doc,subject
0,1,Según el artículo 1.1 de la Constitución Españ...,Un Estado liberal y democrático de Derecho.,Un Estado autonómicoy social de Derecho.,Un Estado social y democrático de Derecho.,Un Estado social y liberal de Derecho.,8679bd3a-c94b-4a41-a2e4-b0c014c28824-ocr.pdf,Constitución Española
1,2,SegúnlaConstitución Española ¿Que tipo de ense...,La enseñanza infantil.,La enseñanza básica.,La formación profesional.,"Toda la educación, excepto la universitaria.",8679bd3a-c94b-4a41-a2e4-b0c014c28824-ocr.pdf,Constitución Española
2,3,Según el Estatuto de Autonomía de la Comunitat...,L'Acadèmia Valenciana de la Llengua.,El Síndic de Greuges.,El Consell Valencià de Cultura.,El Comité Econòmici Social.,8679bd3a-c94b-4a41-a2e4-b0c014c28824-ocr.pdf,Estatut Autonomia
3,4,Según el Estatuto de Autonomía de la Comunitat...,Al Comité Económici Social.,A la Sindicatura de Comptes.,Al Consell Jurídic Consultiu.,Al Síndic de Greuges.,8679bd3a-c94b-4a41-a2e4-b0c014c28824-ocr.pdf,Estatut Autonomia
6,7,Según el artículo7 de la Ley Orgánica 3/2018 d...,Sea mayor decatorce años.,Sea mayor dedieciséis años.,Sea mayor dediecisiete años.,Solo será lícito si consta el consentimiento d...,8679bd3a-c94b-4a41-a2e4-b0c014c28824-ocr.pdf,Protección Datos Personales
...,...,...,...,...,...,...,...,...
1234,38,¿Cuàl de las siguientes afirmaciones forma pan...,Establecer las políticas de contratación de la...,Definir la denominación socialy el objeto soci...,Determinar íasestrategias de marketingy ventas.,Establecer elsegmento de mercado principal al ...,d.txt,Estatutos UPV
1249,56,Según ef artículo 45 de \a Ley Orgánica 2/2023...,"Dos tercios de sus componentes, que incluya, a...","Dos quintos de sus componentes, que incluya, a...","Un quinto de sus componentes, que incluya, al ...","Un tercio de sus componentes, que incluya, al ...",d.txt,LOSU
1272,79,"Segan íaLeyorgánica 3/2007, de 22 de marzo, pa...",hacer efectivo elderecho de igualdad de tratoy...,promover [ascondiciones para que laiguatdad de...,el pleno reconocimiento de la igualdad formal ...,combatir todas lasmanifestaciones de discrimin...,d.txt,Ley 3/2007
1273,80,"La Ley Orgánica 3/2007, de 22 de marzo, para í...",todas laspersonas. D) toda persona jurídica ún...,Con una nota numérica entre0 y 10.,Con una nota numérica entre0 y 5.,"Como “Satisfactorio”, “En proceso”o “No adquir...",d.txt,Ley 3/2007


In [6]:
df.to_csv(ASSETS_DIR.joinpath("preguntas_leyes.csv"), index=False)